# Stable diffusion
V dnešní době si člověk pod pojmem generativní AI představí ChatGPT a jemu podobné modely, tj. modely, které na základě zadání generují text. Nejedná se nicméně o první technologii z této oblasti, která se dostala do širšího povědomí. OpenAI, která má ChatGPT na svědomí, se ještě před jeho uveřejněním pochlubila DALL-E 2, modelem, který dokázal na základě textového zadání vygenerovat velice pěkné obrázky. Od té doby vznikla další verze - DALL-E 3 - a objevili se i konkurenti, z nichž asi nejznámější je Midjourney. Nicméně byly vytvořeny i open-source modely, s jejichž pomocí si člověk může generovat obrázky na svém vlastním stroji. Většina, ne-li všechny takovéto modely, jsou spojeny se Stable Diffusion - rodinou modelů vytvořených společností Stability AI.  
Tento text nemá ambice být do hloubky jdoucím popisem technologie či podrobným návodem, jak s danou technologií pracovat. Na to nechť laskavý čtenář zamíří například [sem](https://stable-diffusion-art.com/). Jedná se spíše o více či méně chaotický soubor mých poznámek. Pokud ale pomohou i někomu dalšímu, budu jen rád.

## Teorie
Před samotnou prací s obrázky si řekněme, co vlastně Stable Diffusion je. Podle definice z [článku](https://arxiv.org/abs/2112.10752) jsou difuzní modely probabilistické modely, které se učí distribuci dat na základě postupného odšumování proměnné. Ugh... co to vlastně znamená?  
Pro pochopení je potřeba vědět, že se stable diffusion modely skládají z dvou podmodelů - jednoho textového a jednoho obrázkového. 
Textový model je natrénován tak, že do něj jdou jak obrázky, tak popisky těchto obrázků. Výstup má v obou případech podobu vektoru významově podobného vektorům z embeddovacích modelů (o embeddovacích modelech více [zde](https://github.com/vitsaidl/workshops/blob/master/text/text.ipynb)). V rámci trénování jsou váhy modelu upravovány tak, aby si byly vektory obrázků a jejich popisků podobné (ve smyslu cosinové podobnosti). V běžném použití pak textový model přebírá popisek a vytváří vektor odpovídající esenci obrázku popisku odpovídající.  
Při přípravě dat pro obrázkový model vezmeme obrázky a aplikujeme na ně definovanou úroveň šumu (tato úroveň je pro různé trénovací obrázky různá). Následně trénujeme model tak, aby dokázal odhadnout míru i rozložení šumu. Při běžném používání pak model krmíme jednak šumem a jednak vektorem z textového modelu, kterýžto vektor charakterizuje nějaký objekt. Ve výsledku pak model z náhodného šumu během několika iterací vyloupne chtěný obrázek.
Fakticky je situace trochu složitější, neboť celá operace neprobíhá v "běžném" prostoru pixelů, nýbrž z důvodu rychlosti a efektivnosti v latentním prostoru. Až na konci obrázkového modelu sedí submodel, který latentní reprezentaci obrázku převádí do pixelů.

## Stable diffusion modely
Stability AI uveřejnila několik svých modelů. Ty byly pak lidmi mimo tuto firmu přetrénovány. Tím pádem existuje hromada různých modelů, z nichž každý má trochu jiné optimální použití.
 - Stable Diffusion 1.5 - "základní" stable diffusion model podporující primárně rozlišení 512x512. Je pro něj asi nejvíc rozširujících LORA modelů (více o pár kapitol níže). Stáhnout model lze například [tady](https://huggingface.co/runwayml/stable-diffusion-v1-5/tree/main) - potřeba je soubor v1-5-pruned-emaonly.safetensors
 - Realistic Vision - model specializující se na realistické obrázky. Jedná se o mutaci Stable Diffusion 1.5, tj. optimální velikost obrázků je taktéž 512x512. Ke stažení [zde](https://civitai.com/models/4201/realistic-vision-v20)
 - Dreamshaper - "portrétní" model sedící na pomezí realistické a počítačové grafiky. Taktéž mutace Stable diffusion 1.5. Ke stažení [tady](https://civitai.com/models/4384/dreamshaper).
 - SDXL - model je stavěn na vyšší rozlišení (1024x1024) a měl by být schopný do obrázků vkládat čitelný text. Bohužel  je i hardwarově náročnější - u ComfyUI je to asi jedno, ale ve Stable Difffusion WebUI s ním kvůli ne úplně ideální optimalizaci pracovat nemohu. Dostupný je například [tady](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/tree/main) (byť já mám zjevně nějakou zmutovanou verzi z civitai).
 - SDXL turbo - model schopný vytvořit obrázky za řádově kratší čas, přičemž na kvalitě to téměř není vidět. Ke stažení [zde](https://huggingface.co/stabilityai/sdxl-turbo/tree/main) - konkrétně jde o soubor sd_xl_turbo_1.0_fp16.safetensors.
 - pro cokoli dalšího je vhodné vlézt na stránky [civitai](https://civitai.com) a vybrat si, co člověk zrovna potřebuje. Jen bacha - je tam dost NSFW obrázků.

## Stable diffusion webui
Stable diffusion webui je asi nejpopulárnějším grafickým rozhraním pro vytváření AI obrázku na vlastním stroji (v konstrastu k vyrábění obrázků skrze webovou službu). Jeho výhodou je poměrně přímočaré ovládání (minimálně v porovnání s ComfyUI). Největší nevýhoda tkví v ne úplně dobré optimalizaci - na svém stroji můžu XL modely rozběhnout s ComfyUI, s Stable diffusion webui ale ne.  
Návod na instalaci nalezneme v [Githubu projektu](https://github.com/AUTOMATIC1111/stable-diffusion-webui). Pro samotné použití pak musíme pouze stáhnout některý z výše zmíněných modelů a umístit ho do adresáře models/Stable-diffusion. Následně už jen do záložky txt2img napíšeme prompt (případně do negativního promptu uvedeme, co nechceme) a klepneme na velké oranžové tlačítko "Generate".  
Níže vidíme příklad toho, co se stane při (ne)použití negativního promptu.
<table>
  <tr>
    <td>
      <img src="sd_images/pos_tree_neg_empty.png"/>
      <p align="center"><em>Pos. prompt: "tree", neg. prompt empty.</em></p>
    </td>
    <td>
      <img src="sd_images/pos_tree_neg_leaves.png"/>
      <p align="center"><em>Pos. prompt: "tree", neg. prompt "leaves".</em></p>
    </td>
  </tr>
</table>
Pakliže chceme nějaké slovo v promptu zdůraznit, obalíme ho do kulatých závorek. Pro jemnější ladění důležitosti slova můžeme za toto slovo (ale stále v kulatých závorkách) napsat dvojtečku a váhu - viz následující příklady.
<table>
  <tr>
    <td>
      <img src="sd_images/apple_tree_with_apples.png"/>
      <p align="center"><em>Apple tree with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket.png"/>
      <p align="center"><em>Apple (tree) with apples</em></p>
    </td>
  </tr>
  <tr>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_0d1.png"/>
      <p align="center"><em>Apple (tree:0.1) with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_0d5.png"/>
      <p align="center"><em>Apple (tree:0.5) with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_1d0.png"/>
      <p align="center"><em>Apple (tree:1.0) with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_1d5.png"/>
      <p align="center"><em>Apple (tree:1.5) with apples</em></p>
    </td>
  </tr>  
  <tr>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_2d0.png"/>
      <p align="center"><em>Apple (tree:2.0) with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_2d5.png"/>
      <p align="center"><em>Apple (tree:2.5) with apples</em></p>
    </td>
    <td>
      <img src="sd_images/apple_tree_with_apples_bracket_7d5.png"/>
      <p align="center"><em>Apple (tree:7.5) with apples</em></p>
    </td>
  </tr>  
</table>

Obecně by měly být prompty co možná nejstručnější. Roli prý hraje i pořadí slov v promptu - slova na začátku mají větší prioritu než slova na konci.

### Generation tab
No jo, ale co znamenají všechna ta hejblátka pod textovými poli pro prompt a negativní prompt?  
<img src="sd_images/txt2img_basic_options.png" width="50%">  
<b>Width</b> a <b>Height</b> jsou asi jasné. Jen bychom měli znovu poznamenat, že modely mají preferovanou velikost obrázků (mutace SD 1.5 512x512, mutace SDXL 1024x1024 apod.) a pokud zvolíme rozměry jiné, může se to na kvalitě výsledku projevit.  
<b>Seed</b> řídí pseudonáhodný generátor šumu. Jde přesně o ten šum, který vstupuje do obrázkového modelu, o kterém jsme mluvili v teorii. Pokud budeme mít stejný prompt, povedou různé hodnoty seedu k různým obrázkům, na druhé straně pro stejný prompt a stejný seed bychom měli dostávat vždy obrázek stejný (pokud jsou tedy stejné i všechny další parametry - například i rozměry obrázku). Jen bacha - hodnota "-1" tu neznamená nějaký konkrétní seed, nýbrž pokyn, že se má seed zvolit náhodně. Vedle seedu vidíme dvě tlačítka - jedno s obrázem kostky a jedno s obrázkem reprezentujícím recyklaci. Kostka změní číslo v poli pro seed na -1, recyklace do pole pro seed vloží seed posledního vygenerovaného obrázku (a to i když jsme seed explicitně nestanovili - program tento údaj totiž najde v metadatech).  
<img src="sd_images/txt2img_extra.png" width="50%">  
Zaklikneme-li checkbox <b>Extra</b>, dostaneme možnost pracovat s <b>Variation seed</b>. Tato věc slouží k vytváření mírných obměn již vytvořeného obrázku. Musíme přitom zvolit i sílu variace - pro 0 nezpozorujeme žádnou změnu, hodnota 1 naopak způsobí, že se vyignoruje původní seed a roli bude hrát pouze variation seed (tj. výsledek bude stejný, jako bychom zaškrtávátko <b>Extra</b> zaškrtnuté neměli a do kolonky <b>Seed</b> vložili hodnotu z <b>Variation seed</b>.  
<b>Resize seed from width/height</b> měly hádám vést k tomu, že při změně rozměru obrázku (a zachování seedu) modifikujeme i rozložení šumu a tak budeme mít ve výsledku stejný obrázek, ale s jinými rozměry. Normálně totiž změna rozměrů vede k odlišnému šumu i při stejném seedu a ve výsledku tak k úplně jinému výsledku. Bohužel tyto položky mi nikdy nefungovaly (ne že bych s nimi běžně pracoval).  
Vraťme se zpátky do hlavního rozhraní a podívejme se na dvojici <b>Batch count</b> a <b>Batch size</b>. <b>Batch size</b> říká, kolik obrázků se může paralelně v jeden okamžik vyrábět. Pokud na to má člověk dostatečně silné GPU, povede to k rychlejšímu generování sady obrázků. Pokud člověk silné GPU nemá, tak uvidí chybovou hlášku na nedostatek paměti. <b>Batch count</b> slouží k specifikování toho, kolik dávek (batchí) obrázků se má (sériově) vyrobit. Ve výsledku tedy například pro hodnoty 1x4, 4x1 či 2x2 dostaneme stejný počet obrázků, ale za jiný čas. Mimochodem, pokud budeme takto generovat více obrázků při specifikovaném seedu, nevytvoří se identické obrázky, nýbrž každý obrázek se vytvoří se seedem o jedna větším, než měl obrázek předchozí.   
<b>Sampling steps</b> specifikuje, v kolika krocích má model povoleno odstraňovat šum. Přitom užitečnost každé další iterace s rostoucím počtem iterací klesá. Tj. obrázky pro počty kroků 1 a 2 se budou dramaticky lišit, obrázky pro počty kroků 101 a 102 ne. Pro normální modely obvykle stačí mít tento parametr někde mezi 30 a 40.  
<table>
  <tr>
    <td>
      <img src="sd_images/sampling_steps_1.png"/>
      <p align="center"><em>Sampling steps = 1</em></p>
    </td>
    <td>
      <img src="sd_images/sampling_steps_5.png"/>
      <p align="center"><em>Sampling steps = 5</em></p>
    </td>
    <td>
      <img src="sd_images/sampling_steps_10.png"/>
      <p align="center"><em>Sampling steps = 10</em></p>
    </td>
    <td>
      <img src="sd_images/sampling_steps_20.png"/>
      <p align="center"><em>Sampling steps = 20</em></p>
    </td>
  </tr>
  <tr>
    <td>
      <img src="sd_images/sampling_steps_30.png"/>
      <p align="center"><em>Sampling steps = 30</em></p>
    </td>
    <td>
      <img src="sd_images/sampling_steps_40.png"/>
      <p align="center"><em>Sampling steps = 40</em></p>
    </td>
    <td>
      <img src="sd_images/sampling_steps_100.png"/>
      <p align="center"><em>Sampling steps = 100</em></p>
    </td>
  </tr>  
</table>
<b>CFG scale</b> (neboli Classifier-Free Guidance scale) ovlivňuje, nakolik má model brát vážně uživatelův prompt. Dalo by se říci, že je tento parametr podobný teplotě u velkých jazykových modelů. Pro malé hodnoty bude model méně šešněrován zadáním a v obrázku se mohou objevit neočekávané věci. Pro velké hodnoty se začne projevovat přílišná saturace barev. Obvykle je vhodné mít tuto veličinu mezi 7 a 15. Na ukázce vidíme vliv různých hodnot <b>CFG scale</b> pro prompt "dog playing with baloon". 
<table>
  <tr>
    <td>
      <img src="sd_images/cfg_1.png" style=""/>
      <p align="center"><em>CFG scale = 1</em></p>
    </td>
    <td>
      <img src="sd_images/cfg_2.png"/>
      <p align="center"><em>CFG scale = 2</em></p>
    </td>
    <td>
      <img src="sd_images/cfg_5.png"/>
      <p align="center"><em>CFG scale = 5</em></p>
    </td>
    <td>
      <img src="sd_images/cfg_10.png"/>
      <p align="center"><em>CFG scale = 10</em></p>
    </td>
  </tr>
  <tr>
    <td>
      <img src="sd_images/cfg_15.png"/>
      <p align="center"><em>CFG scale = 15</em></p>
    </td>
    <td>
      <img src="sd_images/cfg_20.png"/>
      <p align="center"><em>CFG scale = 20</em></p>
    </td>
    <td>
      <img src="sd_images/cfg_30.png"/>
      <p align="center"><em>CFG scale = 30</em></p>
    </td>
  </tr>  
</table>

<b>Sampling method</b> udává způsob, jakým způsobem se bude optimalizovat odstraňování šumu. Nedělám si ambice popsat jednotlivé samplery - zvídavého čtenáře raději odkážu na [tento článek](https://stable-diffusion-art.com/samplers). Jeho take home message je používat <b>DPM++ 2M Karras</b> či <b>PM++ SDE Karras</b> (lepší, ale pomalejší).  
Zaškrtávátko <b>Hires fix</b> se použije, když chceme upscalovat obrázek (tj. zvětšit jeho rozměry) už v procesu jeho vytváření. Tento přístup je vhodnější než měnit <b>Width</b> a <b>Height</b> na hodnoty, se kterými není použitý model kompatibilní. Velikost upscalovaného obrázku určíme buďto šoupátkem <b>Upscale by</b>, anebo parametry <b>Resize width to</b> a <b>Resize height to</b>. Výsledné rozměry by měly být vidět v pravém horních rohu hires. fix sekce.
<b>Hires steps</b> jsou to samé, co <b>Sampling steps</b>. Přitom při hodnotě <b>Hires steps</b> = 0 se fakticky použije číslo ze <b>Sampling steps</b> pro původní obrázek. <b>Denoising strength</b> pak říká, jak moc velká míra šumu se na obrázek přidá, než proběhne "hiresové" odstraňování šumu. Příliš velká hodnota může vést k tomu, že se nový obrázek bude od originálu značně odlišovat.  
Problém s upscalováním navázaným na generování obrázku je v tom, že celý proces trvá příliš dlouho. Když si uvědomíme, že ne každý obrázek se povede, dává smysl upscalování provádět jako separátní akci v sekci <b>Extras</b>.  
<img src="sd_images/txt2img_hires.png" alt="text2img_basic_options" width="50%">  

### Ostatní druhoúrovňové taby
Podívejme se nyní na další položky - taby vedle "Generation".  
<img src="sd_images/sekundarni_zalozka.png" width="50%">  
<b>Textual inversion</b> obsahuje embeddingy. Jedná se o malé soubory (stovky kb) obsahující embedding objektu či stylu, který je SD modelu neznámý. Velice často se používají pro eliminaci nechtěných artefaktů (nerealistické ruce či oči). Zaintegrování embeddingů do GUI probíhá tak, že se (obvykle z [civitai](https://civitai.com/)) stažený soubor umístí do adresáře embeddings a v záložce <b>Textual inversion</b> se klepne na tlačítko <b>refresh</b>. Když chceme pak embedding použít v (negativním) promptu, klikneme do promptovacího textového pole a pak na chtěnou embeddingovou dlaždici v záložce.  
<b>Hypernetworks</b> jsou principielně to samé jako LORA (viz dále) - malé modely, které se nalepí na velký SD model. Rozdíl mezi hypernetwors a lorami  spočívá v geometrii sítě - hypernetworks jsou "normální" fully connected neuronové sítě. Podle internetu to vypadá, že LORy jsou populárnější. Sám jsem hypernetworks nikdy nepoužil, takže k nim moc nemám co dodat.  
V sekci <b>checkpoints</b> nalezneme plnohodnoté (a bohužel i do velikosti, která je v GB) modely.  
<b>LORA</b> modely mají ve své podstatě podobu matic upravující chování mateřského modelu. Na rozdíl od něj nejsou tak velké - jejich velikost činí cca 100 Mb. Nalezneme je stejně jako modely plnohodnotné na [civitai](https://civitai.com/), přičemž po stažení je vložíme do adresáře models/Lora. Použití je téměř identické jako u embeddingů - klepneme do promptového pole a pak na Lora dlaždici. Nicméně do promptu se nevloží pouze jméno Lory, nýbrž i nějaké číslo. Ve výsledku tedy vidíme něco ve stylu <lora:nvdreampunk:1>. Číslo udává, s jakou silou se má LORA aplikovat (a ano, lze uvést i čísla větší než 1 anebo i čísla záporná). Na ukázce vidíme výsledek promptu "cute hamster" s Lorou pro křečky ve stylu tvůrce [yeyuannnnn](https://x.com/yeyuannnnn).
<table>
  <tr>
    <td>
      <img src="sd_images/lora_hamster_0.png" style=""/>
      <p align="center"><em>Síla aplikace LORA modelu: 0</em></p>
    </td>
    <td>
      <img src="sd_images/lora_hamster_0d25.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: 0,25</em></p>
    </td>
    <td>
      <img src="sd_images/lora_hamster_0d5.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: 0,50</em></p>
    </td>
    <td>
      <img src="sd_images/lora_hamster_0d75.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: 0,75</em></p>
    </td>
  </tr>
  <tr>
    <td>
      <img src="sd_images/lora_hamster_1.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: 1</em></p>
    </td>
    <td>
      <img src="sd_images/lora_hamster_2.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: 2</em></p>
    </td>
    <td>
      <img src="sd_images/lora_hamster_m1.png"/>
      <p align="center"><em>Síla aplikace LORA modelu: -1</em></p>
    </td>
  </tr>  
</table>

### Img2img
Přejděme nyní na záložku <b>img2img</b>. Ta se oproti <b>txt2img</b> liší tím, že krom promptu v ní na vstup dáváme i obrázek. V běhu algoritmu pak není na začátku celého procesu náhodný šum, nýbrž zašumněný vložený obrázek. <b>Sketch</b> se od <b>img2img</b> liší tím, že do zdrojového obrázku můžeme kreslit. <b>Inpaint</b> nám dovoluje specifikovat, které části původního obrázku budou SD modelem změněny a které ne. <b>Inpaint sketch</b> by měl být kombinací obojího, ale popravdě netuším, jak přejít od kreslení k výběru masky.
<img src="sd_images/img2img_subtabs.png" width="50%">  
Oproti <b>txt2img</b> se objevuje tu několik dalších parametrů. <b>Denoising strenght</b> má ten samý význam jako u <b>hires fix</b> - jedná se o míru přidavného šumu. V praxi menší číslo = menší přídavný šum = výsledný obrázek se originálu podobá více. Nula povede k tomu, že se v obrázku žádná změna neprovede. Níže se nalézá příklad, kde byl v původním obrázku křeček, avšak my dali do promptu "bunny".
<table>
  <tr>
    <td>
      <img src="sd_images/denoising_0.png" style=""/>
      <p align="center"><em>Denoising = 0</em></p>
    </td>
    <td>
      <img src="sd_images/denoising_0d25.png"/>
      <p align="center"><em>Denoising = 0,25</em></p>
    </td>
    <td>
      <img src="sd_images/denoising_0d5.png"/>
      <p align="center"><em>Denoising = 0,50</em></p>
    </td>
    <td>
      <img src="sd_images/denoising_0d75.png"/>
      <p align="center"><em>Denoising = 0,75</em></p>
    </td>
    <td>
      <img src="sd_images/denoising_1.png"/>
      <p align="center"><em>Denoising = 1</em></p>
    </td>
  </tr>
</table>
<img src="sd_images/inpaint_params.png" width="50%">  
V inpaintingu je ještě několik dalších možností nastavení. <b>Inpaint masked</b>/<b>inpaint not masked</b> říká, zda má model změnit vybranou nebo nevybranou část obrázku. 
<b>Inpaint area</b> je spojená s tím, kvůli čemu se primárně inpainting používá - na opravu nezdařených obličejů u obrázků zachycujících celou postavu člověka. Tyto obličeje jsou nezdařené, neboť v rámci rozlišení původních SD modelů - 512x512 pixelů - ani nemohly v počtu alokovaných pixelů vypadat dobře. Když je <b>inpaint area</b> nastavena na <b>only masked</b>, tak má inpaintovaná oblast plus mínus rozměr 512*512 a až v posledním kroku je downscalovaná, aby se vešla do obrázku. Díky tomu pak obličeje/ruce/malé objekty vypadají lépe. Nebezpečím je ale skutečnost, že inpaintovaná oblast může být nekonzistentní se zbytkem obrázku. U detailů nás to netrápí, pokud ale upravujeme dejme tomu část pozadí, už to problém představuje. V takovém případě pak musíme zvolit nikoli <b>only masked</b>, nýbrž <b>whole picture</b>. V příkladu níže byla k inpaintování vybrána pouze hlava jelena. 
<table>
  <tr>
    <td>
      <img src="sd_images/deer_head_only_masked.png" style=""/>
      <p align="center"><em>Only masked</em></p>
    </td>
    <td>
      <img src="sd_images/deer_head_only_whole_picture.png"/>
      <p align="center"><em>Whole picture</em></p>
    </td>
  </tr>
</table>

<b>Masked content</b> říká, co se má před samotným začátkem práce modelu s inpaintovanou oblastí stát. <b>Original</b> s ní neudělá nic a popravdě je to nejvhodnější varianta. <b>Fill</b> oblast uniformě vyplní barvou v obrázku převažující, <b>latent noise</b> šumem a <b>latent nothing</b> sekci efektivně celou vynuluje (jestli to správně chápu, tak celá operace probíhá v latentním prostoru, kde má toto vynulování asi jinou formu i výsledek než změnu barvy  oblasti na černou/bílou). 
<table>
  <tr>
    <td>
      <img src="sd_images/deer_head_original.png" style=""/>
      <p align="center"><em>Original</em></p>
    </td>
    <td>
      <img src="sd_images/deer_head_fill.png"/>
      <p align="center"><em>Fill</em></p>
    </td>
    <td>
      <img src="sd_images/deer_head_latent_noise.png"/>
      <p align="center"><em>Latent noise</em></p>
    </td>
    <td>
      <img src="sd_images/deer_head_latent_nothing.png"/>
      <p align="center"><em>Latent nothing</em></p>
    </td>
  </tr>
</table>

<b>Mask blur</b> říká, jak velká má být "přechodová" oblast mezi maskou a základním obrázkem. Bez ní by totiž inpaintovaná věc mohla v obrázku působit příliš nepatřičně. Když je ale tato oblast vzhledem k masce příliš velká, není už kam vložit inpaintovaný objekt a tak de facto k impaintování nedojde. K podobnému účelu slouží <b>Only masked padding, pixels</b>. Rozdíl mezi nimi mi popravdě není příliš jasný - možná <b>mask blur</b> efektivně užírá oblast masky, zatímco <b>Only masked padding, pixels</b> ovlivňuje oblast vně masky? V příkladu jsme chtěli do pravého horního rohu inpaintovat havrana.
<table>
  <tr>
    <td>
      <img src="sd_images/blur_0.png" style=""/>
      <p align="center"><em>Mask blur = 0</em></p>
    </td>
    <td>
      <img src="sd_images/blur_20.png"/>
      <p align="center"><em>Mask blur = 20</em></p>
    </td>
  </tr>
</table>

### Extras
Záložka <b>extras</b> slouží k úpravám obrázků. Taby <b>scale by</b> resp <b>scale to</b> určují rozlišení nového obrázku. <b>Upscaler 1</b> pak definuje, podle jakého algoritmu by mělo upscalování probíhat (internet doporučuje použití R-ESRGAN 4x+ pro realistické obrázky, resp. R-ESRGAN 4x+ Anime6B pro kreslené obrázky). Pokud bychom z nějakého důvodu chtěli použít dva upscalery, vybereme druhý v listboxu <b>Upscaler 2</b>. Váhu mezi nimi určí <b>Upscaler 2 visibility</b>.  
<b>GFPGAN</b> slouží na opravu zdeformovaných obličejů. Nicméně když jsem zaškrtl, že ho chci použít, dostal jsem chybu "ValueError: max() arg is an empty sequence". Problém spočíval ve faktu, že v adresáři models/GFPGAN žádný model tohot typu nebyl a ani se nezačal stahovat. Podle rad z internetu by se měl model stáhnout [odsuď](https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth). Stejný účel by měl mít <b>CodeFormer</b>. Nicméně ať už jsem nastavoval parametr <b>visibility</b> jakkoli, žádného rozdílu (snad krom mírné pixelizace) jsem si nevšiml. Proto bude asi lepší pro úpravu obličeje použít <b>img2img</b>.
<b>Auto focal point crop</b> a <b>Auto-sized crop</b> asi měly na základě nějakých parametrů obrázek automaticky ořezávat. Nevšiml jsem si ale, že by k tomu docházelo...  
<b>Caption</b> slouží na vytvoření AI popisku obrázku. Moc přesné to není - obrázek s jelenem a havranem byl popsán slovy
```
against tree, bush, daisy, dandelion, dappled sunlight, day, deer, field, fireflies, flower, foliage, forest, grass, green theme, jungle, light rays, log, moss, mushroom, nature, on grass, open mouth, outdoors, overgrown, path, plant, pond, river, scenery, squirrel, stream, sunbeam, sunlight, tree, tree shade, tree stump, under tree, vines, white flower
```
Navíc je poněkud zvláštní, že výsledný popisek se nezobrazí v GUI, nýbrž ho člověk musí lovit v outputs/extra-images

### Hidden text
Jedná se o více či méně očividné zakomponování masky -  obrázku, kterým může být logo, emotikon, slovo či QR kód - do obrázku normálního.
Postup pro instalaci si už popravdě moc nepamatuji, takže následující řádky berte s rezervou. Napřed je třeba nainstalovat ControlNet rozšíření. To provedeme tak, že překlikneme do záložky <b>Extansion</b> a tam do podzáložky <b>Install from URL</b>. Do <b>URL for extension's git repository</b> zkopírujeme https://github.com/Mikubill/sd-webui-controlnet a klepneme na tlačítko <b>Install</b>.  
Takto jsme nainstalovali rozšíření, nikoli ale potřebný model. Ten stáhneme [odsud](https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/tree/main) - potřebujeme soubor control_v1p_sd15_qrcode_monster.safetensors.  
Pokud se celá operace povedla, měli bychom mít v záložce <b>txt2img</b>, podzáložce <b>Generation</b> novou sekci - ControlNet. V ní by se měl nacházet listbox <b>Models</b>, ve kterém bychom měli nalézt stažený model.  

Použití
  - napíšeme prompt jako obvykle
  - v <b>ControlNet</b> sekci klepneme na <b>Image</b> sekci a vybereme soubor s maskou. Tím se mimo jiné automaticky zatrhne checkbox <b>Enable</b>.
  - pokud jsme to ještě neudělali, vybereme qrcode_monster model
  - klepneme na tlačítko <b>Generate</b>

Ze všech nastavovátek je pro nás důležité asi jen <b>Control Weight</b>. Tento přepínač de facto říká, jak moc má být text skrytý. Pro hodnotu 0 se maska ignoruje, pro hodotu 2 maska dominuje. Podobnou funkci sice můžou mít i tři přepínače v <b>Control Mode</b>, ale tím přijdeme o jemnou kontrolu. Tj. doporučuji nechat tuto možnost na <b>Balanced</b>.  
V příkladu níže byl vytvářen obrázek s promptem "leaves".
<table>
  <tr>
    <td>
      <img src="sd_images/maska_abc.png"/>
      <p align="center"><em>Maska</em></p>
    </td>
    <td>
      <img src="sd_images/hidden_text_control_weight_0d7.png"/>
      <p align="center"><em>Control weight = 0,7</em></p>
    </td>
    <td>
      <img src="sd_images/hidden_text_control_weight_1.png"/>
      <p align="center"><em>Control weight = 1</em></p>
    </td>
    <td>
      <img src="sd_images/hidden_text_control_weight_1d3.png"/>
      <p align="center"><em>Control weight = 1,3</em></p>
    </td>
    <td>
      <img src="sd_images/hidden_text_control_weight_2.png"/>
      <p align="center"><em>Control weight = 2</em></p>
    </td>
  </tr>
</table>

### ControlNet
Co to vlastně je ten ControlNet použitý v předchozím odstavci? Jedná se o neuronovou síť, která přináší nové možnosti pro generování obrázků. Například nám nad rámec textových promptů umožňuje specifikovat výraz tváře či polohu končetin. Text v této kapitole bude poměrně stručný, pokud se tedy o problematice chcete dozvědět více, podívejte se [sem](https://stable-diffusion-art.com/controlnet).  
Dejme tomu, že bychom v našich obrázcích chtěli replikovat tanec slečny [odsud](https://en.wikipedia.org/wiki/Dance#/media/File:La_bailaora_Josefa_Vargas_(1840).jpg). Napřed si musíme stáhnout model detekující postavení těla - OpenPose. Ten najdeme na [tady](https://huggingface.co/lllyasviel/ControlNet-v1-1/tree/main) (jde nám o control_v11p_sd15_openpose.pth). Model zkopírujeme do složky models/ControlNet. Nyní bychom ho měli vidět v controlnetí záložce dropboxu <b>Model</b>. Pokud tomu tak není, klepneme na vedle ležící modré tlačítko <b>Refresh</b>. V dropboxu <b>Preprocessor</b> vybereme <b>openpose</b>. I preprocesor je model, ten se ale stahuje automaticky. Nyní bychom rádi viděli, jestli <b>Preprocessor</b> správně ve vzorovém obrázku detekoval polohu končetin. Klepneme tudíž na červenou hvězdu mezi dropboxy <b>Preprocessor</b> a <b>Model</b>.  
<img src="sd_images/controlnet_settings.png" width="50%"/>  
A... póza nesedí. Klepneme na druhé tlačítko <b>edit</b> v obrázku ji zachycující (první <b>edit</b> by lezl na nějaký webový editor). Napřed smažeme pózu květináče. Ten se skrývá pod <b>Person 2</b>. Klepneme tedy u <b>Person 2</b> na tlačítko <b>X</b> a tím tuto "personu" smažeme. Co se týče špatně chycené ruky u tanečnice, klepneme na barevná kolečka a táhneme, kde chceme odpovídající končetinu mít (to může, ale nemusí korespondovat se zdrojovým obrázkem). Když jsme s výsledkem spokojeni, klepneme na tlačítko <b>Send pose to ControlNet</b>.  
<img src="sd_images/pose_editor.png" width="50%"/>  
Výsledný vygenerovaný obrázek pak může vypadat nějak takto:  
<img src="sd_images/controlnet_dancing_elf.png" width="25%"/> 

Co kdybychom nepoužili <b>OpenPose</b>, ale <b>Canny</b> (model nalezneme na to samém místě, jako OpenPose model)? Tento model nedetekuje pózu člověka, nýbrž okraje objektů v obrázku. Tj. obecný charakter originálního obrázku zůstane zachován.
<img src="sd_images/canny.png"/>
<img src="sd_images/canny_dancing_elf.png" width="25%"/>
<img src="sd_images/dancing_robot_canny.png" width="25%"/>

Pozn.: Jestli vše chápu správně, zaškrtávátka u <b>ControlType</b> slouží pouze k tomu, abychom v dropboxech <b>Processor</b> a <b>Model</b> nemuseli hledat chtěné položky mezi hromadou položek momentálně nechtěných.

### ReActor
[ReActor](https://github.com/Gourieff/sd-webui-reactor) je rozšíření do Stable diffusion webui, které přináší možnost nahrazovat obličeje v obrázcích obličeji z obrázků jiných. Instalace je popsána na stránkách rozšíření, nicméně principielně se jedná o tu samou písničku jako u ControlNetu. Pokud se vše podaří, měli bychom vidět v sekci txt2img novou věc.
<img src="sd_images/reactor_interface.png" width="50%"/>  
Modus operandi je takový, že člověk nahraje do <b>Single source image</b> obrázek s přenášeným obličejem, do normálního promtového pole napíše prompt a pak klikne na <b>Generate</b>. <b>Multiple source images</b> nepovedou k tomu, že by se obličej nějak "zprůměroval", nýbrž se pro každou sem nahranou fotku vytvoří separátní fakový obrázek. Gender detection by měl zajistit, aby ReActor nahrazoval pouze obličeje daného pohlaví, ale spolehnout se na tuto featuru 100% nedá.  
<table>
  <tr>
    <td>
      <img src="sd_images/mad_poet.png">
      <p align="center"><em>Původní vygenerovaný obrázek</em></p>
    </td>
  </tr>
</table>
<b>Restore face</b> je třeba, jinak hrozí "rozpitost" obličeje. Pokud bude <b>Restore Face Visibility</b> na nule, bude to stejné, jako by byl <b>Restore face</b> položen roven <b>None</b>. 
<table>
  <tr>
    <td>
      <img src="sd_images/mad_poet_none_restore_face.png">
      <p align="center"><em>Restore Face nastaven na None</em></p>
    </td>
    <td>
      <img src="sd_images/mad_poet_reactor_vis0_fid0d5.png">
      <p align="center"><em>Restore Face nastaven na None, Restore Face Visibility = 0</em></p>
    </td>
    <td>
      <img src="sd_images/mad_poet_reactor_vis1_fid0d5.png">
      <p align="center"><em>Restore Face nastaven na None, Restore Face Visibility = 1</em></p>
    </td>
  </tr>
</table>

<b>CodeFormer Weight (Fidelity)</b> doporučuji nastavit na 1, kdy je obličej ve výsledném obrázku nejpodobnější nahrazovacímu obličeji. Na příkladech se dívejte na změnu šířky nosu.
<table>
  <tr>
    <td>
      <img src="sd_images/mad_poet_reactor_vis1_fid0.png">
      <p align="center"><em>Fidelity = 0</em></p>
    </td>
    <td>
      <img src="sd_images/mad_poet_reactor_vis1_fid0d5.png">
      <p align="center"><em>Fidelity = 0.5</em></p>
    </td>
    <td>
      <img src="sd_images/mad_poet_reactor_vis1_fid1.png">
      <p align="center"><em>Fidelity = 1</em></p>
    </td>
  </tr>
</table>
Lepší než používat ReActor v txt2img je přejít do img2img, konkrétně do sekce Inpaint. Díky tomu totiž budeme vědět, že základní obrázek je dostatečně kvalitní a můžeme se soustředit jen na obličej. Jen je třeba nezapomenout na stáhnutí <b>Denoising strength</b> na nulu, aby se nám obličej v masce neplánovaně nezměnil tak, že náš nahrazující obličej s ním bude nekompatibilní.    

Pozn.: samozřejmě měnit jdou obličeje i v jiných obrázcích než jen v těch vygenerovaných Stable Diffusion.

## ComfyUI
ComfyUI představuje alternativu k Stable Diffusion WebUI. Jeho výhoda spočívá v lepší optimalizaci a možnosti mít co možná nejvíce věcí pod kontrolou. V tom ale tkví i nevýhoda tohoto prostředí - není tak intuitivní.  

### Instalace
Instalace je popsána [zde](https://github.com/comfyanonymous/ComfyUI). Za pozornost stojí odstavec spojený s úpravou konfiguračního souboru, ve kterém se ComfyUI přikazuje, kde má hledat modely. Jedná se asi o nejsnazší způsob, jak zajistit, aby si je ComFyUI i SD WebUI mohly sdílet. Jelikož se v konfiguráku nastavuje base path, uvidí ComfyUI nejen checkpointy (tj. základní velké modely), ale i lory a embeddingy.    

### Základy ovládání
Po prvním spuštění ComfyUI člověk uvidí základní GUI pro txt2img.  
Celé rozhraní je postavené z pospojovaných nodů. Když se za tuto okolnost přeneseme, máme k dispozici zhruba ty samé ovládací prvky, jako jsme měli u SD WebUI. Kdybychom něčemu nerozuměli, můžeme se podívat do dokumentace - např. stránka k KSampleru se nalézá [tady](https://docs.getsalt.ai/md/Comfy/Nodes/KSampler/?h=ksampler).     
Snad jen jedna poznámka - šipky u každého textového pole by člověka mohly navést k domnění, že se hodnoty v onom poli dají měnit jen s jejich použitím. Není tomu tak - obvykle stačí klepnout do dotčeného pole a objeví se přímá možnost editace.  
A ještě jedna poznámka, která se týká <b>denoise</b> pole v KSampleru, neboť popisek v dokumentaci by nemusel být dostatečný. Tato věcí říká, na kolik procent se má původní obrázek proměnit v rámci odšumování. Jelikož náš "vstupní obrázek" pochází z nodu <b>Empty Latent Image</b>, tj. efektivně v obrázku nic není, máme tu jedničku.  
Asi nemá cenu jít do větších podrobností, když sami tvůrci ComfyUI udělali velice pěkný (byť poněkud... specifický) [tutoriál](https://comfyanonymous.github.io/ComfyUI_tutorial_vn/).
<img src="sd_images/comfyui_intro.png" width="100%"/>  

### Nahrávání workflow
Takto to ještě vypadá jednoduše. Když ale klepneme pravým tlačítkem myši do volného prostoru, uvidíme nabídku nodů. U nich je pak problém na první pohled poznat, k čemu vlastně slouží, a jak by se měly správně do workflow zapojit.  
<img src="sd_images/basic_nodes.png" width="40%"/>  
#### [img2img](https://comfyanonymous.github.io/ComfyUI_examples/img2img/)
Pojďme na to jinak. Zkusme si nahrát dejme tomu workflow pro img2img. To uděláme tak, že [odsud](https://comfyanonymous.github.io/ComfyUI_examples/img2img/img2img_workflow.png) stáhneme obrázek flow. Následně obrázek workflow nahraje skrze tlačítko <b>Load</b> na panelu vlevo dole. Jak je to vůbec možné? Funguje to díky skutečnosti, že stažený obrázek obsahuje metadata o workflow. Tato metadata obsahuje i každý obrázek v ComfyUI vyrobený. Tj. když nahrajeme obrázek vytvoření s txt2img workflow, vrátí se nám toto workflow zpět na obrazovku. Nicméně nahrávat (a ukládat) takto můžeme i čisté jsony. 
Proberme si pár dalších zajímavých workflow.
#### [inpainting](https://comfyanonymous.github.io/ComfyUI_examples/inpaint/)  
Pro vyznačení oblasti, kterou chceme změnit, klepneme na načtený obrázek v nodu <b>Load image and alpha mask for inpainting</b> pravým tlačítkem myši a vybereme <b>Open in MaskEditor</b>. Nutno podotknout, že flow dle všeho počítá s použitím dedikovaného inpaintovacího modelu. Pokud ho nechcete stahovat, je třeba nahradit node <b>VAE Encode (for inpainting)</b> za <b>VAE Encode</b> (k nalezení v nabídce pravého myšítka <b>Add node</b> > <b>latent</b>) následovaný <b>Set latent noise mask</b> (<b>Add node</b> > <b>latent</b> > <b>inpaint</b>) - více [zde](https://stable-diffusion-art.com/inpaint-comfyui/). Ale popravdě - asi něco dělám špatně, neboť výsledek je pořád horší než to, co dokážu udělat ve SD WebUI.
#### [outpainting](https://comfyanonymous.github.io/ComfyUI_examples/inpaint/yosemite_outpaint_example.png)  
Principielně stejné jako předchozí případ, ale máme tu o jeden node navíc. V něm nastavíme, o kolik (a jakým směrem) se má obrázek zvětšit + jak ostrá má být hranice mezi původním obrázkem a outpaintovanými částmi (parametr <b>feathering</b>).  
#### [unCLIP](https://comfyanonymous.github.io/ComfyUI_examples/unclip/)
UnCLIP slouží k tomu, abychom styl nějakého obrazu/fotky předali modelem vytvářenému obrázku. Efektivně tak dáváme SD modelu na vstup nějaký obrázek se zadáním "Udělej mi něco v tomhle stylu".  
Pro použití unCLIPu je třeba použít dedikovaný model - já jsem šáhnul po [tomto](https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/tree/main).  
UnCLIP je primárně řízen nodem <b>unCLIPconditioning</b>. <b>Strength</b> říká, jako moc velký vliv na výsledek bude mít vstupní obrázek. [Definici](https://blenderneko.github.io/ComfyUI-docs/Core%20Nodes/Conditioning/unCLIPConditioning/) <b>noise_augmentation</b> moc nerozumím (v kontextu dejme tomu seedu - možná je to efektivně jemné ladění v rámci jedné seed hondoty?)) - každopádně čím je toto větší, tím odlišnější obrázek člověk dostává. Níže vidíme výsledky promptu "tundra in the morning", přičemž jako vstupní obrázek byla použita "Starry night" od van Gogha.
<table>
  <tr>
    <td>
      <img src="sd_images/tundra_strength0.png">
      <p align="center"><em>strength = 0.00, noise_augmentation = 0.3</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_strength25.png">
      <p align="center"><em>strength = 0.25, noise_augmentation = 0.3</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_strength50.png">
      <p align="center"><em>strength = 0.50, noise_augmentation = 0.3</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_strength75.png">
      <p align="center"><em>strength = 0.75, noise_augmentation = 0.3</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_strength100.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.3</em></p>
    </td>
  </tr>
  <tr>
    <td>
      <img src="sd_images/tundra_noise0.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.0</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_noise10.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.1</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_noise20.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.2</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_noise40.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.4</em></p>
    </td>
    <td>
      <img src="sd_images/tundra_noise60.png">
      <p align="center"><em>strength = 1.00, noise_augmentation = 0.6</em></p>
    </td>
  </tr>
</table>
Zmiňme nakonec, že jako vstup můžeme mít i více než jeden obrázek.

#### [LORA](https://comfyanonymous.github.io/ComfyUI_examples/lora/)  
LORU zapneme tak, že mezi <b>Load checkpoint</b> a <b>CLIP Text Encode (Prompt)</b> vložíme node <b>Load LORA</b> (k nalezení v <b>loaders</b>).

#### [Embeddings](https://comfyanonymous.github.io/ComfyUI_examples/textual_inversion_embeddings/)
Pro embeddingy žádný speciální node neexistuje. Stačí do <b>CLIP Text Encode (Prompt)</b> napsat
```
embedding:BadDream
```
či 
```
embedding:BadDream.pt
```
Tj. přípona embedding souboru je nepovinná.  

#### [Upscalers](https://comfyanonymous.github.io/ComfyUI_examples/upscale_models/)  
Pro upscalování obrázků potřebujeme do flow přidat <b>Load upscale model</b> (k nalezení v <b>loaders</b>) a <b>Upscale Image (using Model)</b> (<b>image</b> > <b>upscaling</b>). Do druhého zmíněného půjde jak výstup nodu prvního, tak výstup <b>VAE decode</b>.